In [1]:
# -*- coding: utf-8 -*-
# Block 1: Model Initialization and Resource Loading
import torch
import torch.nn.functional as F
import numpy as np
import json
import os
from PIL import Image
from sentence_transformers import SentenceTransformer
import ipywidgets as widgets
from IPython.display import display, clear_output
import io # Required for BytesIO

# --- 匯入自訂模組 ---
# 請確保 main.py, model_components.py, data_utils.py, losses.py 在同一個目錄
# 或者已經正確安裝為 Python 模組

try:
    # 從 main.py (或您獨立定義這些函數的檔案)
    from main import NumpyEncoder, process_label_tensor, calculate_average_metrics, display_average_results, compare_ablation_results, gen_setting_name, apply_setting
    # 從 model_components.py
    from model_components import MultiCPRFL, VisualSemanticInteraction, MultiPromptInitializer, FiLMFusion, ContrastiveProjector
    # 從 data_utils.py
    from data_utils import load_index, convert_to_multihot, attributes # 假設 attributes 列表在這裡定義
    # 從 losses.py (雖然推論用不到，但 MultiCPRFL 可能內部有引用，保留以防萬一)
    from losses import get_loss_function, get_contrastive_loss, PromptSeparationLoss, calculate_class_weights
except ImportError as e:
    print(f"導入自訂模組時發生錯誤: {e}")
    print("請確保 main.py, model_components.py, data_utils.py, losses.py 都在正確的路徑下。")
    # 你可以在這裡停止執行或提供備用方案
    raise e # 直接拋出錯誤停止執行

# --- 全域變數與設定 ---
MODEL_PATH = "Best_model/model_VSI=True_Fusion=film_Loss=asl_CLT=image_text_v2_L=1.0-1.0_B=1.0_K=None_PB=0.999_VB=1536_TB=1536_rep4_38.61.pth"
INDEX_PATH = "SymCat.json" # 假設您的索引檔案名稱是這個
CLIP_MODEL_NAME = "jinaai/jina-clip-v2"

# --- 輔助函數 ---
def initialize_multi_category_embeddings(attributes, idx_to_label_dict, clip_model, device, index_data):
    """
    根據每個屬性的所有類別，生成對應的 CLIP 文字嵌入。
    idx_to_label_dict: {attr: {idx: label_id}, ...}
    index_data: SymCat.json 載入後的資料
    """
    # 模板取自您的訓練腳本輸出
    prompt_templates = {
        'crop': "a photo of {} plant, showing plant features and characteristics",
        'part': "closeup of plant {}, a specific part of the plant",
        'symptomCategories': "plant disease showing {} symptoms, a category of plant disease signs",
        'symptomTags': "plant disease with {}, a specific symptom on plant"
    }

    # 從 index_data 建立 ID 到描述的映射
    descriptions = {}
    try:
        for item in index_data.get('crops', []): descriptions[str(item['id'])] = item.get('description', str(item['id']))
        for item in index_data.get('parts', []): descriptions[str(item['id'])] = item.get('description', str(item['id']))
        for category in index_data.get('symptomCategories', []):
            cat_id_str = str(category['id'])
            cat_desc = category.get('description', cat_id_str)
            # 嘗試提取英文描述
            if '(' in cat_desc and ')' in cat_desc: eng_desc = cat_desc.split('(')[1].rstrip(')')
            else: eng_desc = cat_desc
            descriptions[cat_id_str] = eng_desc
            for tag in category.get('tags', []):
                tag_id_str = str(tag['id'])
                tag_desc = tag.get('description', tag_id_str)
                if '(' in tag_desc and ')' in tag_desc: eng_tag_desc = tag_desc.split('(')[1].rstrip(')')
                else: eng_tag_desc = tag_desc
                descriptions[tag_id_str] = eng_tag_desc
    except Exception as e:
        print(f"從 index_data 提取描述時出錯: {e}")
        # 即使出錯也嘗試繼續，後面會有檢查

    embeddings_dict = {}
    for attr in attributes:
        if attr not in idx_to_label_dict:
            print(f"警告: 屬性 '{attr}' 不在 idx_to_label_dict 中，跳過。")
            continue

        # 使用 idx_to_label_dict 的 keys (int or str) 來獲取 ID 列表
        # 並確保 ID 已轉換為字串以匹配 descriptions 的鍵
        sorted_indices = sorted(idx_to_label_dict[attr].keys())
        text_prompts = []
        missing_desc_ids = []
        valid_categories = [] # 儲存實際生成嵌入的類別 ID (以 idx_to_label 的 key 為準)

        for idx in sorted_indices:
            label_id = idx_to_label_dict[attr][idx] # 獲取原始 ID
            label_id_str = str(label_id) # 確保是字串鍵

            if label_id_str in descriptions:
                desc = descriptions[label_id_str]
                if attr in prompt_templates:
                     text = prompt_templates[attr].format(desc)
                else: # Fallback
                    text = f"photo of {desc}"
                text_prompts.append(text)
                valid_categories.append(label_id) # 記錄這個ID有生成prompt
            else:
                missing_desc_ids.append(label_id_str)
                # 選擇性地，如果描述缺失，可以跳過或使用預設文本
                # text_prompts.append(prompt_templates[attr].format(f"ID {label_id_str}")) # 使用 ID 作為備用

        if missing_desc_ids:
             print(f"警告: 屬性 '{attr}' 找不到以下 ID 的描述: {missing_desc_ids[:5]}{'...' if len(missing_desc_ids) > 5 else ''}")

        if not text_prompts:
            print(f"警告: 屬性 '{attr}' 沒有有效的文字提示詞可生成嵌入，跳過。")
            continue

        # 生成嵌入
        with torch.no_grad():
            clip_emb = clip_model.encode(text_prompts, convert_to_tensor=True, batch_size=128) # 使用 convert_to_tensor
        clip_emb = F.normalize(clip_emb, p=2, dim=-1)
        embeddings_dict[attr] = clip_emb.to(device).float()
        print(f"屬性 '{attr}' 的文字嵌入已生成，形狀: {embeddings_dict[attr].shape}")

    return embeddings_dict

def extract_image_features(image, clip_model, device):
    """
    使用 Jina-CLIP 模型提取圖像特徵
    """
    try:
        if image.mode != 'RGB':
            image = image.convert('RGB')
        with torch.no_grad():
            # Jina CLIP 直接編碼 PIL Image
            image_features = clip_model.encode(image, convert_to_tensor=True).to(device).float()
            image_features = F.normalize(image_features, dim=0) # Jina輸出可能是1D，正規化
            if image_features.dim() == 1:
                image_features = image_features.unsqueeze(0) # 確保有 batch 維度 [1, D]
        return image_features
    except Exception as e:
        print(f"提取圖像特徵時出錯: {e}")
        raise e

def load_model_and_resources(model_path, index_path):
    """
    載入 MultiCPRFL 模型、CLIP 模型及相關資源。
    """
    global attributes # 使用 data_utils 中的全局 attributes 列表

    # 設置裝置
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"使用裝置: {device}")

    # 檢查檔案存在
    if not os.path.exists(model_path):
        print(f"錯誤: 找不到模型權重檔案: {model_path}")
        return None
    if not os.path.exists(index_path):
        print(f"錯誤: 找不到索引檔案: {index_path}")
        return None

    # 載入索引檔案
    try:
        label_to_idx_dict, idx_to_label_dict, index_data = load_index(index_path)
        print(f"成功載入索引檔案: {index_path}")
    except Exception as e:
        print(f"載入索引檔案時出錯: {e}")
        return None

    # 初始化 Jina-CLIP 模型
    try:
        print(f"載入 Jina-CLIP 模型 ({CLIP_MODEL_NAME})...")
        # 指定 torch_dtype=torch.float32 避免 Jina CLIP 預設的 bfloat16 (如果在不支援的硬體上)
        clip_model = SentenceTransformer(CLIP_MODEL_NAME, trust_remote_code=True, model_kwargs={"torch_dtype": torch.float32})
        clip_model = clip_model.to(device)
        clip_model.eval()
        print("Jina-CLIP 模型載入完成。")
    except Exception as e:
        print(f"載入 Jina-CLIP 模型時出錯: {e}")
        return None

    # --- 推斷模型超參數 (根據您的訓練腳本和模型名稱) ---
    # 這些需要與您訓練時使用的參數 *完全一致*
    prompt_dim = 1536           # 從 VB=1536, TB=1536 推斷
    text_embed_dim = 1024       # Jina CLIP 輸出維度
    hidden_dim = prompt_dim // 2 # 768
    image_input_dim = 1024      # Jina CLIP 圖像輸出維度
    use_vsi = "VSI=True" in model_path
    fusion_mode = "film"        # 從 Fusion=film 推斷
    use_contrastive = True      # 假設使用了對比學習
    dropout_rate = 0.2          # 假設值，最好與訓練時一致
    vsi_num_heads = 8           # 假設值，最好與訓練時一致
    visual_bottleneck_dim = 1536 # 從 VB=1536 推斷
    text_bottleneck_dim = 1536   # 從 TB=1536 推斷

    print("\n--- 模型初始化參數 ---")
    print(f"Attributes: {attributes}")
    print(f"Prompt Dim: {prompt_dim}")
    print(f"Hidden Dim: {hidden_dim}")
    print(f"Text Embed Dim: {text_embed_dim}")
    print(f"Image Input Dim: {image_input_dim}")
    print(f"Use VSI: {use_vsi}")
    print(f"Fusion Mode: {fusion_mode}")
    print(f"Visual Bottleneck Dim: {visual_bottleneck_dim}")
    print(f"Text Bottleneck Dim: {text_bottleneck_dim}")
    print("-----------------------\n")


    # 計算每個屬性的類別數量
    try:
        num_classes_dict = {attr: len(label_to_idx_dict[attr]) for attr in attributes}
    except KeyError as e:
        print(f"錯誤: 屬性 '{e}' 不在 label_to_idx_dict 中，請檢查索引檔案或 attributes 列表。")
        return None

    # 初始化 MultiCPRFL 模型
    try:
        print("初始化 MultiCPRFL 模型...")
        model = MultiCPRFL(
            attributes=attributes,
            text_embed_dim=text_embed_dim,
            prompt_dim=prompt_dim,
            hidden_dim=hidden_dim,
            num_classes_dict=num_classes_dict,
            image_input_dim=image_input_dim,
            use_vsi=use_vsi,
            fusion_mode=fusion_mode,
            use_contrastive=use_contrastive, # 啟用 contrastive projector (如果模型有)
            dropout_rate=dropout_rate,
            vsi_num_heads=vsi_num_heads,
            visual_bottleneck_dim=visual_bottleneck_dim,
            text_bottleneck_dim=text_bottleneck_dim
        ).to(device)
        print("MultiCPRFL 模型結構初始化完成。")
    except Exception as e:
        print(f"初始化 MultiCPRFL 模型時出錯: {e}")
        import traceback
        traceback.print_exc()
        return None

    # 載入預訓練的模型權重
    try:
        print(f"載入模型權重從: {model_path}")
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.eval() # 設定為評估模式
        print(f"成功載入模型權重。")
    except Exception as e:
        print(f"載入模型權重時出錯: {e}")
        print("請檢查模型路徑是否正確，以及模型定義是否與權重檔匹配。")
        return None

    # 預先生成文字嵌入 (優化，避免每次預測都重新生成)
    try:
        print("正在預先生成所有類別的文字嵌入...")
        # 注意：這裡需要 idx_to_label_dict 來知道每個索引對應哪個 ID
        text_embeddings_dict = initialize_multi_category_embeddings(attributes, idx_to_label_dict, clip_model, device, index_data)
        print("文字嵌入生成完成。")
    except Exception as e:
        print(f"生成文字嵌入時出錯: {e}")
        return None

    return {
        'model': model,
        'clip_model': clip_model,
        'index_data': index_data,
        'label_to_idx_dict': label_to_idx_dict,
        'idx_to_label_dict': idx_to_label_dict,
        'text_embeddings_dict': text_embeddings_dict, # 儲存預先生成的嵌入
        'device': device,
        'attributes': attributes # 將 attributes 列表也存入
    }

# --- 執行初始化 ---
print("--- 開始載入模型與資源 ---")
MODEL_RESOURCES = load_model_and_resources(MODEL_PATH, INDEX_PATH)

if MODEL_RESOURCES:
    print("\n--- 初始化完成 ---")
    print("MODEL_RESOURCES 變數已準備好，可以執行下一個區塊進行預測。")
else:
    print("\n--- 初始化失敗 ---")


--- 開始載入模型與資源 ---
使用裝置: cuda
成功載入索引檔案: SymCat.json
載入 Jina-CLIP 模型 (jinaai/jina-clip-v2)...


C:\Users\admin\.cache\huggingface\modules\transformers_modules\jinaai\jina-clip-implementation\51f02de9f2cf8afcd3bac4ce996859ba96f9f8e9\modeling_clip.py:140: UserWarning: Flash attention is not installed. Check https://github.com/Dao-AILab/flash-attention?tab=readme-ov-file#installation-and-features for installation instructions, disabling
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Jina-CLIP 模型載入完成。

--- 模型初始化參數 ---
Attributes: ['crop', 'part', 'symptomCategories', 'symptomTags']
Prompt Dim: 1536
Hidden Dim: 768
Text Embed Dim: 1024
Image Input Dim: 1024
Use VSI: True
Fusion Mode: film
Visual Bottleneck Dim: 1536
Text Bottleneck Dim: 1536
-----------------------

初始化 MultiCPRFL 模型...
[MultiCPRFL] Visual Bottleneck Dim: 1536
  [MultiPromptInitializer] Text Bottleneck Dim: 1536
MultiCPRFL 模型結構初始化完成。
載入模型權重從: Best_model/model_VSI=True_Fusion=film_Loss=asl_CLT=image_text_v2_L=1.0-1.0_B=1.0_K=None_PB=0.999_VB=1536_TB=1536_rep4_38.61.pth
成功載入模型權重。
正在預先生成所有類別的文字嵌入...
屬性 'crop' 的文字嵌入已生成，形狀: torch.Size([3, 1024])
屬性 'part' 的文字嵌入已生成，形狀: torch.Size([8, 1024])
屬性 'symptomCategories' 的文字嵌入已生成，形狀: torch.Size([15, 1024])
屬性 'symptomTags' 的文字嵌入已生成，形狀: torch.Size([79, 1024])
文字嵌入生成完成。

--- 初始化完成 ---
MODEL_RESOURCES 變數已準備好，可以執行下一個區塊進行預測。


In [ ]:
# -*- coding: utf-8 -*-
# Block 2: Prediction Interface and Execution (Show Description)

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from PIL import Image
import torch
import torch.nn.functional as F
import numpy as np
import io
import base64
import traceback # Import traceback for better error printing

# --- 檢查 MODEL_RESOURCES 是否已成功載入 ---
if 'MODEL_RESOURCES' not in globals() or MODEL_RESOURCES is None:
    print("錯誤：模型資源尚未初始化。請先成功執行上一個程式碼區塊。")
    display(widgets.HTML("<h3 style='color:red'>錯誤：模型資源尚未初始化。請先成功執行上一個程式碼區塊。</h3>"))
else:
    print("模型資源已載入，準備建立上傳介面...")

    # --- 輔助函數 (從 Block 1 移過來或確保可用) ---
    def extract_image_features(image, clip_model, device):
        try:
            if image.mode != 'RGB':
                image = image.convert('RGB')
            with torch.no_grad():
                image_features = clip_model.encode(image, convert_to_tensor=True).to(device).float()
                image_features = F.normalize(image_features, dim=0)
                if image_features.dim() == 1:
                    image_features = image_features.unsqueeze(0)
            return image_features
        except Exception as e:
            print(f"提取圖像特徵時出錯: {e}")
            raise e

    def get_description_from_id(label_id, index_data):
        """從 index_data 中查找 ID 對應的 description"""
        label_id_str = str(label_id) # 確保使用字串比較
        # 檢查 crops
        for item in index_data.get('crops', []):
            if str(item.get('id')) == label_id_str:
                return item.get('description', label_id_str)
        # 檢查 parts
        for item in index_data.get('parts', []):
            if str(item.get('id')) == label_id_str:
                return item.get('description', label_id_str)
        # 檢查 symptomCategories 和 symptomTags
        for category in index_data.get('symptomCategories', []):
            if str(category.get('id')) == label_id_str:
                return category.get('description', label_id_str)
            for tag in category.get('tags', []):
                if str(tag.get('id')) == label_id_str:
                    return tag.get('description', label_id_str)
        # 如果都找不到，返回原始 ID
        print(f"警告: 在 index_data 中找不到 ID '{label_id_str}' 的描述。")
        return label_id_str

    # --- 預測與顯示函數 ---
    def predict_image_pil(image, model_resources, top_k=10):
        """
        使用載入的資源預測 PIL 圖片結果，並包含描述。
        """
        model = model_resources['model']
        clip_model = model_resources['clip_model']
        idx_to_label_dict = model_resources['idx_to_label_dict']
        text_embeddings_dict = model_resources['text_embeddings_dict']
        device = model_resources['device']
        attributes = model_resources['attributes']
        index_data = model_resources['index_data'] # 需要 index_data 來找描述

        print(f"正在分析上傳的圖片...")

        # 1. 提取圖片特徵
        try:
            if not isinstance(image, Image.Image):image = Image.open(image)
            image_features = extract_image_features(image, clip_model, device)
        except Exception as e:
            print(f"無法處理圖片或提取特徵: {e}")
            return None

        # 2. 進行預測
        model.eval()
        with torch.no_grad():
            output = model(image_features, text_embeddings_dict)
            if isinstance(output, tuple) and len(output) >= 2:
                logits_dict, _ = output[0], output[1] # 忽略 final_prompts_dict
            else:
                logits_dict, _ = output

            # 處理預測結果
            results = {}
            for attr in attributes:
                if attr not in logits_dict or attr not in idx_to_label_dict or attr not in text_embeddings_dict:
                    print(f"警告: 屬性 '{attr}' 的 logits 或映射不存在，跳過。")
                    continue

                probs = torch.sigmoid(logits_dict[attr]).cpu().numpy()[0]
                class_probs_with_desc = [] # 修改變數名稱
                num_classes_for_attr = len(idx_to_label_dict[attr])

                if len(probs) != num_classes_for_attr:
                     print(f"警告: 屬性 '{attr}' 的預測維度 ({len(probs)}) 與類別數 ({num_classes_for_attr}) 不符，跳過。")
                     continue

                for idx in range(num_classes_for_attr):
                    if idx in idx_to_label_dict[attr]:
                        label_id = idx_to_label_dict[attr][idx]
                        # --- *** 從 index_data 獲取描述 *** ---
                        description = get_description_from_id(label_id, index_data)
                        # -------------------------------------
                        prob_value = probs[idx] if idx < len(probs) else 0.0
                        # --- *** 返回的 tuple 中加入 description *** ---
                        class_probs_with_desc.append((idx, label_id, description, float(prob_value)))
                        # -----------------------------------------
                    # else: pass

                # 按概率降序排序
                class_probs_with_desc.sort(key=lambda x: x[3], reverse=True) # 按概率排序 (索引3)
                top_predictions = class_probs_with_desc[:top_k]
                results[attr] = top_predictions

        print("圖片分析完成。")
        return results

    def display_predictions_widget(predictions):
        """
        使用 widgets 顯示預測結果表格 (顯示 description)
        """
        if not predictions:
            return widgets.HTML(value="<h3 style='color:red'>沒有預測結果可以顯示</h3>")

        items_to_display = []
        for attr, top_preds in predictions.items():
            title_html = widgets.HTML(f"<h3 style='margin-top: 15px; color: #1f618d;'>{attr} Top {len(top_preds)} 預測</h3>")
            items_to_display.append(title_html)
            header_html = """
            <table style='border-collapse: collapse; width: 95%; border: 1px solid #bdc3c7; margin-bottom: 15px;'>
                <tr style='background-color: #ecf0f1; font-weight: bold;'>
                    <th style='border: 1px solid #bdc3c7; padding: 8px; text-align: center; width: 10%;'>排名</th>
                    <th style='border: 1px solid #bdc3c7; padding: 8px; text-align: left; width: 60%;'>描述</th>
                    <th style='border: 1px solid #bdc3c7; padding: 8px; text-align: right; width: 30%;'>信心度</th>
                </tr>
            """
            rows_html = ""
            # --- *** 使用 description 而不是 class_id *** ---
            for i, (_, _, description, prob) in enumerate(top_preds, 1): # 忽略索引和ID
                 bg_color = "#ffffff" if i % 2 != 0 else "#fdfefe"
                 rows_html += f"""
                 <tr style='background-color: {bg_color};'>
                     <td style='border: 1px solid #bdc3c7; padding: 8px; text-align: center;'>{i}</td>
                     <td style='border: 1px solid #bdc3c7; padding: 8px;'>{description}</td>
                     <td style='border: 1px solid #bdc3c7; padding: 8px; text-align: right;'>{prob*100:.2f}%</td>
                 </tr>
                 """
            # --- *************************************** ---
            table_html = header_html + rows_html + "</table>"
            items_to_display.append(widgets.HTML(table_html))
        return widgets.VBox(items_to_display)

    def create_upload_interface(model_resources):
        """
        創建上傳按鈕和相關的輸出區域
        """
        upload_button = widgets.FileUpload(
            accept='image/*',
            multiple=False,
            description='選擇圖片上傳',
            button_style='info',
            layout=widgets.Layout(width='auto')
        )
        
        # 修改這一行，移除所有樣式設定，不要有框框和滾動條
        image_output = widgets.Output()
        
        spinner_html = """
        <div class="lds-ring" style="display: none; margin: 10px auto;"><div></div><div></div><div></div><div></div></div>
        <style>.lds-ring {display: inline-block;position: relative;width: 40px;height: 40px;}.lds-ring div {box-sizing: border-box;display: block;position: absolute;width: 32px;height: 32px;margin: 4px;border: 4px solid #3498db;border-radius: 50%;animation: lds-ring 1.2s cubic-bezier(0.5, 0, 0.5, 1) infinite;border-color: #3498db transparent transparent transparent;}.lds-ring div:nth-child(1) {animation-delay: -0.45s;}.lds-ring div:nth-child(2) {animation-delay: -0.3s;}.lds-ring div:nth-child(3) {animation-delay: -0.15s;}@keyframes lds-ring {0% {transform: rotate(0deg);}100% {transform: rotate(360deg);}}</style>
        """
        spinner = widgets.HTML(spinner_html)
        results_output = widgets.Output()

        def show_spinner(show=True):
            if show: spinner.value = spinner.value.replace("display: none", "display: block")
            else: spinner.value = spinner.value.replace("display: block", "display: none")

        def resize_image(img, max_dim=500):
            w, h = img.size
            if max(w, h) <= max_dim: return img
            scale = max_dim / max(w, h)
            return img.resize((int(w * scale), int(h * scale)), Image.LANCZOS)

        def on_upload_change(change):
            uploaded_value = upload_button.value
            if not uploaded_value: return

            image_output.clear_output()
            results_output.clear_output()

            uploaded_file_info = None
            if isinstance(uploaded_value, tuple):
                if len(uploaded_value) > 0: uploaded_file_info = uploaded_value[0]
            elif isinstance(uploaded_value, dict):
                 if len(uploaded_value) > 0: uploaded_file_info = next(iter(uploaded_value.values()))

            if uploaded_file_info is None:
                 with results_output: display(widgets.HTML("<h3 style='color:red'>未能解析文件信息</h3>"))
                 return

            try:
                file_content = uploaded_file_info['content']
                file_name = uploaded_file_info['name']
            except KeyError as e:
                 with results_output: display(widgets.HTML(f"<h3 style='color:red'>無法讀取文件內容或名稱: 缺少鍵 {e}</h3>"))
                 return
            except Exception as e:
                 with results_output: display(widgets.HTML(f"<h3 style='color:red'>提取文件信息時出錯: {e}</h3>"))
                 return

            with image_output:
                print(f"載入圖片: {file_name}")
                try:
                    img = Image.open(io.BytesIO(file_content))
                    display_img = resize_image(img.copy())
                    
                    # 使用 base64 直接嵌入 HTML 顯示圖片，不添加任何邊框或背景
                    buffered = io.BytesIO()
                    display_img.save(buffered, format="PNG")
                    img_str = base64.b64encode(buffered.getvalue()).decode()
                    display(HTML(f'<img src="data:image/png;base64,{img_str}" style="border:none;background:none;" />'))
                except Exception as e:
                    print(f"無法顯示圖片: {e}")
                    return

            with results_output:
                show_spinner(True)
                try:
                    predictions = predict_image_pil(img, model_resources)
                    show_spinner(False)
                    if predictions:
                        display(display_predictions_widget(predictions))
                    else:
                        display(widgets.HTML("<p style='color:orange;'>分析完成，但未得到有效預測結果。</p>"))
                except Exception as e:
                    show_spinner(False)
                    print(f"預測過程中發生錯誤: {e}")
                    traceback.print_exc()
                    display(widgets.HTML(f"<p style='color:red;'>分析失敗: {e}</p>"))

            # 清空 FileUpload
            if isinstance(upload_button.value, dict): upload_button.value.clear()
            elif isinstance(upload_button.value, tuple): upload_button.value = ()
            upload_button._counter = 0

        upload_button.observe(on_upload_change, names='value')

        interface_title = widgets.HTML("<h2>Multi-Attribute Plant Analysis</h2><p>Upload an image to get predictions for different attributes.</p>")
        return widgets.VBox([
            interface_title,
            upload_button,
            # 直接使用 image_output，不包裹在 HBox 中，也不加標籤
            image_output,
            spinner,
            results_output
        ])

    # --- 建立並顯示介面 ---
    upload_interface = create_upload_interface(MODEL_RESOURCES)
    display(upload_interface)